In [ ]:
import os
import json
from collections import defaultdict

def coco2yolo_bbox(bbox, img_w, img_h):
    x, y, w, h = bbox
    x_center = x + w / 2
    y_center = y + h / 2
    return [
        x_center / img_w,
        y_center / img_h,
        w / img_w,
        h / img_h
    ]

def convert_coco_json_to_yolo(json_path, img_dir, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    with open(json_path, 'r') as f:
        data = json.load(f)
    images = {img['id']: img for img in data['images']}
    cats = sorted({ann['category_id'] for ann in data['annotations']})
    cat2idx = {cat: i for i, cat in enumerate(cats)}
    imgid2annos = defaultdict(list)
    for ann in data['annotations']:
        imgid2annos[ann['image_id']].append(ann)
    for img_id, img in images.items():
        file_name = img['file_name']
        img_w, img_h = img['width'], img['height']
        label_path = os.path.join(out_dir, os.path.splitext(file_name)[0] + '.txt')
        with open(label_path, 'w') as f:
            for ann in imgid2annos.get(img_id, []):
                class_id = cat2idx[ann['category_id']]
                yolo_box = coco2yolo_bbox(ann['bbox'], img_w, img_h)
                f.write(f"{class_id} {' '.join([str(round(x,6)) for x in yolo_box])}\n")
    print(f"已生成YOLO标签txt到 {out_dir}")
    # 生成data.yaml
    with open('data.yaml', 'w') as f:
        f.write(f"train: {os.path.abspath(img_dir)}\n")
        f.write(f"val: {os.path.abspath(img_dir)}\n")
        f.write(f"nc: {len(cats)}\n")
        f.write(f"names: {cats}\n")
    print("已生成data.yaml")

# 用法
convert_coco_json_to_yolo('train.json', 'images', 'labels')

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8s.pt')  # 加载预训练模型
model.train(
    data='data.yaml',
    imgsz=300,
    epochs=50,
    batch=8
)

Ultralytics 8.3.156 🚀 Python-3.12.3 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 3090, 24135MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=300, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profi

100%|██████████| 5.35M/5.35M [00:00<00:00, 13.6MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[300] must be multiple of max stride 32, updating to [320]
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2554.7±830.4 MB/s, size: 184.2 KB)


train: Scanning /root/autodl-tmp/cowboydetection/labels... 3062 images, 1817 backgrounds, 0 corrupt: 100%|██████████| 4879/4879 [00:03<00:00, 1530.04it/s]


train: New cache created: /root/autodl-tmp/cowboydetection/labels.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1474.3±1108.6 MB/s, size: 290.9 KB)


val: Scanning /root/autodl-tmp/cowboydetection/labels.cache... 3062 images, 1817 backgrounds, 0 corrupt: 100%|██████████| 4879/4879 [00:00<?, ?it/s]
Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (8)

Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 8 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50     0.676G       1.67      2.511      1.457         19        320: 100%|██████████| 610/610 [00:57<00:00, 10.67it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 305/305 [00:17<00:00, 17.64it/s]


                   all       4879       5594       0.66      0.276      0.213      0.108

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/50     0.887G      1.749      2.167      1.534          6        320:  95%|█████████▍| 578/610 [00:53<00:01, 16.22it/s]